In [0]:
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    col, lit, sin, cos, pi, lag, avg,
    max as spark_max, min as spark_min, expr, when, concat,
    round # 현재 코드에는 없지만 이전 코드에서 라운딩을 사용했다면 필요
    # substring, split, regexp_replace, sum 등은 현재 코드에 사용되지 않으므로 제거
    # unix_timestamp, from_unixtime, concat_ws 등은 현재 코드에서 더 이상 직접 사용되지 않으므로 제거
)
from pyspark.sql.window import Window

In [0]:
df=spark.read.option("header", "true").csv('/Volumes/project/default/project/join_v2.csv/')

In [0]:
df.groupBy("year").count().orderBy("year").show()

In [0]:
display(df.limit(5))

In [0]:
# 'year', 'month', 'day', 'hour' 컬럼을 조합하여 timestamp 컬럼 생성
df = df.withColumn('timestamp_str',
                   concat(
                       col('year'), lit('-'),
                       col('month'), lit('-'),
                       col('day'), lit(' '),
                       expr("LPAD(hour, 2, '0')"), lit(':00:00')
                   )) \
       .withColumn('timestamp', col('timestamp_str').cast('timestamp')) \
       .drop('timestamp_str') # 임시 컬럼 제거

In [0]:
# 중요: 시계열 처리를 위해 District와 timestamp 기준으로 정렬해야 합니다.
df = df.orderBy(col('District'), col('timestamp'))

In [0]:
# 기존 월별 주기성 피처 대신, 단순 월 컬럼 또는 각 월을 나타내는 이진형 피처 고려
# 여름철 데이터에 한정되므로 month_sin/cos의 중요도 낮음.
# 여기서는 'month' 컬럼을 그대로 사용하고, 'is_july_august' 같은 특정 피크 월 이진형 피처를 추가해 봅니다.
df = df.withColumn('hour_sin', sin(2 * pi() * col('hour') / 24)) \
       .withColumn('hour_cos', cos(2 * pi() * col('hour') / 24)) \
       .withColumn('day_of_week', expr("dayofweek(timestamp)")) # PySpark dayofweek는 일요일=1, 토요일=7
df = df.withColumn('day_of_week_sin', sin(2 * pi() * (col('day_of_week') -1) / 7))
df = df.withColumn('day_of_week_cos', cos(2 * pi() * (col('day_of_week') -1) / 7))
# 야간 시간대 이진형 피처 추가 (열섬 심화)
df = df.withColumn('is_night_time', when((col('hour') >= 21) | (col('hour') < 6), 1).otherwise(0))

In [0]:
# Window 명세 정의: District별로 그룹화하고 timestamp 기준으로 정렬 (필수!)
window_spec = Window.partitionBy('District').orderBy('timestamp')

# 1) 현재 교외 온도
df = df.withColumn('suburban_temp_current', col('avg_temp_aws'))

# 2) 24시간 전 교외 온도 (lag_8)
df = df.withColumn('suburban_temp_lag_24hr', lag(col('avg_temp_aws'), 8).over(window_spec))

# 3) 지난 24시간 이동 평균 교외 온도 (rolling_avg_24hr)
df = df.withColumn('suburban_temp_ma_24hr', avg(col('avg_temp_aws')).over(window_spec.rowsBetween(-8, -1)))

# 4) 3시간 전 교외 온도 (lag_1)
df = df.withColumn('suburban_temp_lag_3hr', lag(col('avg_temp_aws'), 1).over(window_spec))

# UHII (레이블) 지연 피처도 추가 (XGBoost 입력 피처로 사용)
df = df.withColumn('UHII_lag_8', lag(col('UHII'), 8).over(window_spec))
df = df.withColumn('UHII_lag_1', lag(col('UHII'), 1).over(window_spec)) # 단기 UHII 변화도 중요

In [0]:
# --- 필요 없는 원본 컬럼 제거 ---
# 'year', 'day', 'hour'는 'timestamp'와 파생된 주기성 피처로 대체되었으므로 제거
# 'avg_temp_aws'는 'suburban_temp_current' 등으로 대체되었으므로 제거
# 'avg_temp_sdot'는 UHII 계산에 사용되었으므로 제거
# 'day_of_week'는 sin/cos 변환되었으므로 제거
df = df.drop('month', 'day', 'hour', 'avg_temp_sdot', 'avg_temp_aws', 'day_of_week')

In [0]:
# --- NaN 값 처리 ---
df_processed = df.na.drop()

In [0]:
print(f"최종 DataFrame 컬럼: {df_processed.columns}")
print(f"최종 DataFrame 크기: {df_processed.count()} 행")

In [0]:
display(df_processed.limit(1))

In [0]:
df_processed.printSchema()

In [0]:
df=df_processed
output_path = "/Volumes/project/default/project/UHI_data_test.csv"
df.coalesce(1).write.mode("overwrite").option("header", "true").csv(output_path)

print(f"DataFrame이 '{output_path}' 경로에 성공적으로 덮어쓰여졌습니다.")

In [0]:
df=df_processed

df.groupBy("year").count().orderBy("year").show()